In [1]:
# Import statements
import pandas as pd

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("punkt")
stop_words = stopwords.words()
from tqdm import tqdm

[nltk_data] Downloading package stopwords to C:\Users\HENRY
[nltk_data]     OKEOMA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\HENRY
[nltk_data]     OKEOMA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df1 = pd.read_csv(r"C:\Users\HENRY OKEOMA\Downloads\e commerce reviews train.csv")

In [3]:
df2 = pd.read_csv(r"C:\Users\HENRY OKEOMA\Downloads\e commerce reviews test.csv")

In [4]:
# Bring the unique Labels
df1['labels'].unique()

array(['__label__2', '__label__1'], dtype=object)

In [5]:
# Check for the row at index 6
df1.iloc[6]['text']

'Buyer beware: This is a self-published book, and if you want to know why--read a few paragraphs! Those 5 star reviews must have been written by Ms. Haddon\'s family and friends--or perhaps, by herself! I can\'t imagine anyone reading the whole thing--I spent an evening with the book and a friend and we were in hysterics reading bits and pieces of it to one another. It is most definitely bad enough to be entered into some kind of a "worst book" contest. I can\'t believe Amazon even sells this kind of thing. Maybe I can offer them my 8th grade term paper on "To Kill a Mockingbird"--a book I am quite sure Ms. Haddon never heard of. Anyway, unless you are in a mood to send a book to someone as a joke---stay far, far away from this one!'

In [6]:
#  Mapping the values by using dictionaries \old value and new value

mapping_values = {
    "__label__1" : "negative",
    "__label__2" : "positive"
}

In [7]:
# Mapping the Labels on train data
df1['labels'] = df1['labels'].map(mapping_values)

In [8]:
# Mapping the Labels on test data
df2['labels'] = df2['labels'].map(mapping_values)

In [9]:
# Lets see the new DataFrame
df1.head(8)

,labels,text
0,positive,Stuning even for the non-gamer: This sound tra...
1,positive,The best soundtrack ever to anything.: I'm rea...
2,positive,Amazing!: This soundtrack is my favorite music...
3,positive,Excellent Soundtrack: I truly like this soundt...
4,positive,"Remember, Pull Your Jaw Off The Floor After He..."
5,positive,an absolute masterpiece: I am quite sure any o...
6,negative,"Buyer beware: This is a self-published book, a..."
7,positive,Glorious story: I loved Whisper of the wicked ...


In [10]:
# Remove the stopwords, the bag of words and tfidf
text = "i love this product, it is very good"

In [11]:
# Tokenise the text. split into list of words
words = nltk.word_tokenize(text)

In [12]:
# Filtering out stopwords
filtered_words = [word for word in words if word not in stop_words]
filtered_words

['love', 'product', ',']

In [13]:
# Reconstruct the list to a sentence using join method
filtered_text = " ".join(filtered_words)

In [14]:
def remove_stopwords(text):
  """
  this function takes in a setence
  tokenises the sentence
  filter out stopwords and returns a more compact sentence
  """
  words = nltk.word_tokenize(text)
  filtered_words = [word for word in words if word not in stop_words]
  filtered_text = " ".join(filtered_words)
  return filtered_text

In [15]:
remove_stopwords(text)

'love product ,'

In [16]:
df1['text'].head(10).apply(remove_stopwords)

0    Stuning non-gamer : This sound track beautiful...
1    The soundtrack . : I 'm reading lot reviews 'g...
2    Amazing ! : This soundtrack favorite music tim...
3    Excellent Soundtrack : I soundtrack I enjoy vi...
4    Remember , Pull Your Jaw Off The Floor After H...
5    absolute masterpiece : I taking time read play...
6    Buyer beware : This self-published book , -- r...
7    Glorious story : I loved Whisper wicked saints...
8    A FIVE STAR BOOK : I finished reading Whisper ...
9    Whispers Wicked Saints : This easy read book m...
Name: text, dtype: object

In [17]:
total_rows = len(df1)
tqdm.pandas(total = total_rows)
df1['stop words'] = df1['text'].progress_apply(remove_stopwords)

100%|█████████████████████████████████████████████████████████████████████| 3600010/3600010 [10:10:34<00:00, 98.27it/s]


In [18]:
total_rows = len(df2)
tqdm.pandas(total = total_rows)
df2['stop words'] = df2['text'].progress_apply(remove_stopwords)

100%|████████████████████████████████████████████████████████████████████████| 400000/400000 [1:20:24<00:00, 82.91it/s]


In [20]:
# Creat Bag of words and Create tf-idf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [21]:
### Creating Bag of words from stopwords column
vectorizer = CountVectorizer()
train_bow = vectorizer.fit_transform(df1['stop words'])
test_bow = vectorizer.transform(df2['stop words'])

In [23]:
### Creating a tf-idf matrix from stopwords
tfidf_vectorizer = TfidfVectorizer()
train_tfidf = tfidf_vectorizer.fit_transform(df1['stop words'])

test_tfidf = tfidf_vectorizer.transform(df2['stop words'])

### MODELLING

In [ ]:
# Vader on normal sentences
# Vader on sent6ences withyout stop words
# Custom on train_tfidf
# Custom on train_bow

In [24]:
# Vader on normal sentences
import nltk
nltk.download("vader_lexicon")

# From Vader import sentiment analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to C:\Users\HENRY
[nltk_data]     OKEOMA\AppData\Roaming\nltk_data...


In [25]:
# Vader is a pretrained model for analysing sentiment of sentences
analyzer = SentimentIntensityAnalyzer()

In [34]:
example_text = "i love the orange flavour, good product"

In [35]:
sentiment_scores = analyzer.polarity_scores(example_text)

In [36]:
compound_score = sentiment_scores['compound']
compound_score

0.7964

In [37]:
def analyze_sentence(sentence):
  """
  takes a sentence
  get the sentiments score using analyzer
  return positive if compound score greater tahn 0, else return negative
  """
  sentiment_scores = analyzer.polarity_scores(sentence)
  compound_score = sentiment_scores['compound']
  if compound_score > 0:
    return "positive"
  else:
    return "negative"

In [38]:
analyze_sentence(example_text)

'positive'

In [39]:
# Vader on text
df2['vader_on_text'] = df2['text'].apply(analyze_sentence)

In [40]:
# Vader on stopwords column
df2['vader_on_text_without_stopwords'] = df2['stop words'].apply(analyze_sentence)

In [41]:
# training customes model on bag of words and tf-idf
train_tfidf

<3600010x980461 sparse matrix of type '<class 'numpy.float64'>'
	with 110767474 stored elements in Compressed Sparse Row format>

In [42]:
train_bow

<3600010x980461 sparse matrix of type '<class 'numpy.int64'>'
	with 110767474 stored elements in Compressed Sparse Row format>

In [43]:
# Multinomial Naive bayes
from sklearn.naive_bayes import MultinomialNB

In [44]:
# Creating Classifier objects for tfidf and bow model
classifier_bow = MultinomialNB()
classifier_tfidf = MultinomialNB()

In [45]:
# Fit the models
classifier_bow.fit(train_bow, df1['labels'])
classifier_tfidf.fit(train_tfidf, df1['labels'])

MultinomialNB()

In [46]:
test_bow


<400000x980461 sparse matrix of type '<class 'numpy.int64'>'
	with 12237613 stored elements in Compressed Sparse Row format>

In [47]:
test_tfidf

<400000x980461 sparse matrix of type '<class 'numpy.float64'>'
	with 12237613 stored elements in Compressed Sparse Row format>

In [48]:
# making predictions with classifier

df2['bow'] = classifier_bow.predict(test_bow)

In [49]:
df2['tfidf'] = classifier_bow.predict(test_tfidf)

### MODEL EVALUATION: USING ACCURACY SCORES AND CLASSIFICATION REPORT

In [50]:
from sklearn.metrics import accuracy_score, classification_report

In [51]:
# Evaluation
df2.head(5)

,labels,text,stop words,vader_on_text,vader_on_text_without_stopwords,bow,tfidf
0,positive,Great CD: My lovely Pat has one of the GREAT v...,Great CD : My lovely Pat GREAT voices generati...,positive,positive,positive,positive
1,positive,One of the best game music soundtracks - for a...,One game music soundtracks - game I n't play :...,positive,positive,positive,positive
2,negative,Batteries died within a year ...: I bought thi...,Batteries died year ... : I bought charger Jul...,positive,positive,negative,negative
3,positive,"works fine, but Maha Energy is better: Check o...","works fine , Maha Energy : Check Maha Energy '...",positive,positive,negative,negative
4,positive,Great for the non-audiophile: Reviewed quite a...,Great non-audiophile : Reviewed bit combo play...,positive,positive,positive,positive


In [52]:
vader_text_accuracy_score = accuracy_score(df2['labels'], df2['vader_on_text'])

In [53]:
print(classification_report(df2['labels'], df2['vader_on_text']))

              precision    recall  f1-score   support

    negative       0.87      0.51      0.64    200000
    positive       0.65      0.92      0.76    200000

    accuracy                           0.72    400000
   macro avg       0.76      0.72      0.70    400000
weighted avg       0.76      0.72      0.70    400000



In [54]:
vader_text_stopwords_accuracy_score = accuracy_score(df2['labels'], df2['vader_on_text_without_stopwords'])

In [55]:
print(classification_report(df2['labels'], df2['vader_on_text_without_stopwords']))

              precision    recall  f1-score   support

    negative       0.83      0.55      0.66    200000
    positive       0.66      0.89      0.76    200000

    accuracy                           0.72    400000
   macro avg       0.74      0.72      0.71    400000
weighted avg       0.74      0.72      0.71    400000



In [56]:
bow_score = accuracy_score(df2['labels'], df2['bow'])

In [57]:
print(classification_report(df2['labels'], df2['bow']))

              precision    recall  f1-score   support

    negative       0.84      0.85      0.85    200000
    positive       0.85      0.84      0.85    200000

    accuracy                           0.85    400000
   macro avg       0.85      0.85      0.85    400000
weighted avg       0.85      0.85      0.85    400000



In [58]:
tfidf_score = accuracy_score(df2['labels'], df2['tfidf'])

In [59]:
print(classification_report(df2['labels'], df2['tfidf']))

              precision    recall  f1-score   support

    negative       0.82      0.84      0.83    200000
    positive       0.84      0.82      0.83    200000

    accuracy                           0.83    400000
   macro avg       0.83      0.83      0.83    400000
weighted avg       0.83      0.83      0.83    400000



### CONCLUSION

In [71]:
def inference(text):
    # remove stopwords, convert remaining text to bag of words, and pass
    # the bag of words to classifier_bow, then return predictions

    filtered_text = remove_stopwords(text)

    
    bow_single = vectorizer.transform([filtered_text])  

    inference_result = classifier_bow.predict(bow_single)

    return inference_result[0]


In [72]:
example = 'i love the orange flavour, it is so good'

In [73]:
inference(example)

'positive'